In [ ]:
import json
from typing import NamedTuple


from kfp import dsl
from kfp.v2 import compiler
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (
    component,
    InputPath,
    OutputPath,
    Input,
    Output,
    Artifact,
    Dataset,
    Model,
    ClassificationMetrics,
    Metrics,
)

from kfp.v2.google.client import AIPlatformClient
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [ ]:
from view_demo.utils import env_vars as evar
csv_path = evar.DATASET_CSV
print(csv_path)

PROJECT_ID = evar.PROJECT_ID
REGION = evar.REGION
CONTAINER_URI = evar.BASE_IMAGE_URI
SRC_ROOT = evar.SRC_ROOT
PIPELINE_ROOT = evar.PIPELINE_ROOT

In [ ]:
@component(
    base_image=CONTAINER_URI,
    output_component_file=f'{SRC_ROOT}/preprocess/preprocess.yaml',
)
def view_preprocess(
    project_id: str,
    raw_dataset: str,
    out_dataset: OutputPath(),
):
    from view_demo.preprocess import create_dataset
    bq_path = create_dataset(project_id=project_id, csv_path=raw_dataset)
    with open(out_dataset, 'w') as f:
        f.write(bq_path)
    

In [ ]:
@dsl.pipeline(
    name="view-test-pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def view_pipeline(
    project_id: str = PROJECT_ID,
    raw_dataset: str = csv_path,
):
    import logging
    preprocess_task = view_preprocess(project_id=project_id, raw_dataset=raw_dataset)


In [ ]:
from kfp.v2 import compiler as v2compiler
v2compiler.Compiler().compile(pipeline_func=view_pipeline,
                              package_path='view_pipeline_spec.json')

In [ ]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(
    project_id=PROJECT_ID, 
    region=REGION, 
    )

In [ ]:
result = api_client.create_run_from_job_spec(
    job_spec_path="view_pipeline_spec.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
    parameter_values={
    },
)